In [8]:
import requests

In [85]:
category = 'entertainment'

In [86]:
response = requests.get("http://newsapi.org/v2/top-headlines?country=in&category="+category+"&apiKey=469e3b5dc7b44bf6ae90867fe0e8fd5c")

In [87]:
data = response.json()

In [88]:
from types import SimpleNamespace
import json

In [89]:
articles = data['articles']

In [90]:
news = []
for article in articles:
    title_parts = article['title'].split(" - ")
    text = title_parts[0] + article['description']
    news.append(text)

In [52]:
import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())
import pickle

tokenizer_path = Path('../datasets/sentiment_analysis/tokenizer.pickle').resolve()
with tokenizer_path.open('rb') as file:
    tokenizer = pickle.load(file)

from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model

input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = 4
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

model_weights_path = Path('../models/emotion_recognition/model_weights.h5').resolve()
model.load_weights(model_weights_path.as_posix())

import pandas as pd

encoder_path = Path('../models/emotion_recognition/encoder.pickle').resolve()
with encoder_path.open('rb') as file:
    encoder = pickle.load(file)
    
from nlp import preprocess
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np


In [53]:
subs = "There was an idea, to bring together a group of remarkable people, to see if we could become something more. So when they needed us we could fight the battles that they never could. In time, you'll know what it's like to lose. To feel so desperately that you're right, that to fail all the same. Dread it, Run from it. Destiny still arrives. Evacuate the city. Engage all defences and get this man a shield. Fun isn't something one considers the balance in the universe. But this does put a smile on my face. Who the hell are you guys?"
dialogs = subs.split(". ")

In [54]:
selected = dialogs[9]
sequences = [text.split() for text in selected]
list_tokenized = tokenizer.texts_to_sequences(sequences)
x_data = pad_sequences(list_tokenized, maxlen=100)

y_pred = model.predict(x_data)
emo = dict()
for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
    print(encoder.classes_[index] + ": " + str(value))
    emo[encoder.classes_[index]]=value

anger: 0.13067813
fear: 0.7585651
joy: 0.021767307
sadness: 0.08898952


In [55]:
min_diff = 5
ans = 0
for i in range(len(news)):
    cleaned_data = news[i]
    sequences = [text.split() for text in cleaned_data]
    list_tokenized = tokenizer.texts_to_sequences(sequences)
    x_data = pad_sequences(list_tokenized, maxlen=100)

    y_pred = model.predict(x_data)
    
    cur_diff = 0
    for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
        cur_diff += abs(emo[encoder.classes_[index]] - value)

    if cur_diff < min_diff:
        ans = i
        min_diff = cur_diff


In [56]:
print(news[ans] + " : " + selected)

Horoscope today: December 27, 2020Vogue Daily Horoscope, 27th December 2020: Today's prediction for all zodiac signs: Aries, Taurus, Gemini, Cancer, Leo, Virgo. Libra, Scorpio, Sagittarius, Capricorn, Aquarius, Pisces. Check out our daily horoscope for free : But this does put a smile on my face


In [57]:
cleaned_data = news[1]
sequences = [text.split() for text in cleaned_data]
list_tokenized = tokenizer.texts_to_sequences(sequences)
x_data = pad_sequences(list_tokenized, maxlen=100)

y_pred = model.predict(x_data)
for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
    print(encoder.classes_[index] + ": " + str(value))

anger: 0.13265857
fear: 0.7518926
joy: 0.021220021
sadness: 0.09422911


In [99]:
import cv2 

def generateResult(videoClip,text, dialog):
    cap = cv2.VideoCapture(videoClip) 

    frame_width = int(cap.get(3)) 
    frame_height = int(cap.get(4)) 

    size = (frame_width, frame_height) 
   
    # Below VideoWriter object will create 
    # a frame of above defined The output  
    # is stored in 'filename.avi' file. 
    out = cv2.VideoWriter('Meme_'+videoClip[:8] + text[:8]+ '.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))


    while(True): 

        # Capture frames in the video 
        ret, frame = cap.read() 
        if ret == False:
            break



        # describe the type of font 
        # to be used. 
        font = cv2.FONT_HERSHEY_SIMPLEX 

        # Use putText() method for 
        # inserting text on video 

        cv2.putText(frame,  
                    text,  
                    (50, 50),  
                    font, 0.8,  
                    (100, 250, 230),  
                    2,  
                    cv2.LINE_4) 
        
        cv2.putText(frame,  
                    dialog,  
                    (50, 150),  
                    font, 0.7,  
                    (0, 255, 255),  
                    2,  
                    cv2.LINE_4) 

        # Display the resulting frame 
        cv2.imshow('resultOutput', frame) 
        out.write(frame)

        # creating 'q' as the quit  
        # button for the video 
        if cv2.waitKey(22) & 0xFF == ord('q'): 
            break

    # release the cap object 
    cap.release() 
    # close all windows 
    cv2.destroyAllWindows() 
    out.release()

    

In [100]:
def insert_newlines(string, every=64):
    return '\n'.join(string[i:i+every] for i in range(0, len(string), every))

In [101]:
def getMeme(dialogs,news):
    movie_line = 0
    pairs = dict()
    global_diff = 6
    for j in range(len(dialogs)):
        selected = dialogs[j]
        sequences = [text.split() for text in selected]
        list_tokenized = tokenizer.texts_to_sequences(sequences)
        x_data = pad_sequences(list_tokenized, maxlen=100)

        y_pred = model.predict(x_data)
        emo = dict()
        for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
            print(encoder.classes_[index] + ": " + str(value))
            emo[encoder.classes_[index]]=value

        min_diff = 3
        news_line = 0
        for i in range(len(news)):
            cleaned_data = news[i]
            sequences = [text.split() for text in cleaned_data]
            list_tokenized = tokenizer.texts_to_sequences(sequences)
            x_data = pad_sequences(list_tokenized, maxlen=100)

            y_pred = model.predict(x_data)

            cur_diff = 0
            for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
                cur_diff += abs(emo[encoder.classes_[index]] - value)

            if cur_diff < min_diff:
                news_line = i
                min_diff = cur_diff

        if min_diff < global_diff:
            movie_line = j
            global_diff = min_diff
            
        disp = articles[news_line]['title'] + " : " + dialogs[j]
        print(articles[news_line]['title'] + " : " + dialogs[j] )
        text = insert_newlines(disp,40)
        generateResult(dialogs[j][:8] + '.mp4', articles[news_line]['title'], dialogs[j] )
        pairs[j] = news_line
    
    #print("Final Output --> "+ news[pairs[movie_line]] + " : " + dialogs[movie_line])


In [102]:
getMeme(dialogs,news)

anger: 0.13230877
fear: 0.75031745
joy: 0.020764206
sadness: 0.0966099
Kangana Ranaut’s first director Anurag Basu claims he doesn’t understand her ‘public persona’ - Times of India : There was an idea, to bring together a group of remarkable people, to see if we could become something more
anger: 0.13240752
fear: 0.75724274
joy: 0.022118635
sadness: 0.08823136
'Get well soon, Soorya': Mammootty wishes ‘Thalapathi’ Rajini a speedy recovery - The News Minute : So when they needed us we could fight the battles that they never could
anger: 0.13177775
fear: 0.7610933
joy: 0.022968303
sadness: 0.0841608
Celebrate Christmas With Alia Bhatt: Have A Look How It's Done - IWMBuzz : In time, you'll know what it's like to lose
anger: 0.13157234
fear: 0.7553917
joy: 0.021886934
sadness: 0.09114935
See Pic: ‘Cheers to the last weekend of the year’, Priyanka Chopra bids adieu to 2020 - Times of India : To feel so desperately that you're right, that to fail all the same
anger: 0.13294794
fear: 0.75034